In [1]:
import pandas as pd
import numpy as np
from gestor_datos_climaticos import GestorDatosClimaticos
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import torch
from torch import device
from modelo_lstm import LSTM
import warnings
warnings.filterwarnings("ignore")


In [2]:
# Inicializar el gestor de base de datos
ciudad = 'Piura'
gestor_db = GestorDatosClimaticos()
id_ciudad = gestor_db.obtener_id_ciudad_por_nombre(ciudad)
df_clima = gestor_db.obtener_dataframe('clima', id_ciudad)

In [3]:
# Convertir la columna 'time' a tipo datetime y extraer mes y día
df_clima['time'] = pd.to_datetime(df_clima['time'])
df_clima['mes'], df_clima['dia'] = df_clima['time'].dt.month, df_clima['time'].dt.day

# Determinar la estación del año
def determinar_estacion(mes, dia):
    if (mes == 12 and dia >= 1) or mes in [1, 2]: return 'Verano'
    if mes in [3, 4, 5]: return 'Otoño'
    if mes in [6, 7, 8]: return 'Invierno'
    if mes in [9, 10, 11]: return 'Primavera'

df_clima['estacion'] = df_clima.apply(lambda row: determinar_estacion(row['mes'], row['dia']), axis=1)

# Crear columnas indicadoras para estaciones y meses
estaciones_dummies = pd.get_dummies(df_clima['estacion'], drop_first=False).astype(int)
meses_dummies = pd.get_dummies(df_clima['mes'], prefix='mes').astype(int)

# Agregar las nuevas columnas al DataFrame original
df = pd.concat([df_clima, estaciones_dummies, meses_dummies], axis=1)

mean_temp_min = df['tmin'].mean()
df['tmin'] = df['tmin'].apply(lambda x: mean_temp_min if x < 10 else x)

In [4]:
# Definir las características de entrada y las variables de salida
features = [col for col in df.columns if col not in ['id_ciudad', 'time', 'snow', 'wpgt', 'tsun','estacion']]

features_lstm = [col for col in df.columns if col not in ['id_ciudad', 'time', 'snow', 'wpgt', 'tsun','estacion'
                                                     ,'Primavera','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12']] 
target_features = ['tmax', 'tmin', 'tavg']

In [5]:
# Dividir los datos en entrenamiento, validación y testeo
df_train = df[df['time'] <= '2023-12-31']
df_val = df[(df['time'] > '2023-12-31') & (df['time'] <= '2024-06-30')]
df_test = df[df['time'] > '2024-06-30']
print(f'La data de entrenamiento tiene {len(df_train)} filas')
print(f'La data de validación tiene {len(df_val)} filas')
print(f'La data de testeo tiene {len(df_test)} filas')

La data de entrenamiento tiene 1991 filas
La data de validación tiene 182 filas
La data de testeo tiene 190 filas


In [6]:
# Preparación de datos de entrada para modelos no LSTM
X_train = df_train[features].values
X_val = df_val[features].values
X_test = df_test[features].values

#Entradas LSTM
X_train_lstm = df_train[features_lstm].values 
X_val_lstm = df_val[features_lstm].values
X_test_lstm = df_test[features_lstm].values

# Preparación de datos de salida para modelos
y_train = df_train[target_features].values
y_val = df_val[target_features].values
y_test = df_test[target_features].values

In [7]:
# Configuración de parámetros de ventana temporal
VENTANA_TIEMPO = 25  # T pasos de tiempo
DIMENSION_ENTRADA = len(features)  # Número de características
N_ENTRENAMIENTO = len(df_train) - VENTANA_TIEMPO
N_VALIDACION = len(df_val) - VENTANA_TIEMPO
N_TESTEO = len(df_test) - VENTANA_TIEMPO
DIMENSION_SALIDA = 3  # Número de salidas

In [8]:
# Normalización de datos (excepto LSTM)
normalizador = StandardScaler()
normalizador.fit(X_train)

# Normalización de los conjuntos de datos
X_train = normalizador.transform(X_train)
X_val = normalizador.transform(X_val)
X_test = normalizador.transform(X_test)

# Preparación de datos de entrenamiento con ventana temporal
X_entrenamiento = np.zeros((N_ENTRENAMIENTO, VENTANA_TIEMPO, DIMENSION_ENTRADA))
y_entrenamiento = np.zeros((N_ENTRENAMIENTO, DIMENSION_SALIDA))

for t in range(N_ENTRENAMIENTO):
    X_entrenamiento[t, :, :] = X_train[t:t + VENTANA_TIEMPO]
    y_entrenamiento[t] = y_train[t + VENTANA_TIEMPO]

# Preparación de datos de validación con ventana temporal
X_validacion = np.zeros((N_VALIDACION, VENTANA_TIEMPO, DIMENSION_ENTRADA))
y_validacion = np.zeros((N_VALIDACION, DIMENSION_SALIDA))

for i in range(N_VALIDACION):
    t = i
    X_validacion[i, :, :] = X_val[t:t + VENTANA_TIEMPO]
    y_validacion[i] = y_val[t + VENTANA_TIEMPO]

# Preparación de datos de testeo con ventana temporal
X_testeo = np.zeros((N_TESTEO, VENTANA_TIEMPO, DIMENSION_ENTRADA))
y_testeo = np.zeros((N_TESTEO, DIMENSION_SALIDA))

for i in range(N_TESTEO):
    t = i
    X_testeo[i, :, :] = X_test[t:t + VENTANA_TIEMPO]
    y_testeo[i] = y_test[t + VENTANA_TIEMPO]

In [9]:
import joblib
normalizador_lstm = joblib.load('escalado.pkl')
DIMENSION_ENTRADA = len(features_lstm)  # Número de características

# Normalizar las entradas LSTM
X_train_lstm = normalizador_lstm.transform(df_train[features_lstm].values)
X_val_lstm = normalizador_lstm.transform(df_val[features_lstm].values)
X_test_lstm = normalizador_lstm.transform(df_test[features_lstm].values)

# Preparación de datos de entrenamiento con ventana temporal
X_entrenamiento_lstm = np.zeros((N_ENTRENAMIENTO, VENTANA_TIEMPO, DIMENSION_ENTRADA))

for t in range(N_ENTRENAMIENTO):
    X_entrenamiento_lstm[t, :, :] = X_train_lstm[t:t + VENTANA_TIEMPO]

# Preparación de datos de validación con ventana temporal
X_validacion_lstm = np.zeros((N_VALIDACION, VENTANA_TIEMPO, DIMENSION_ENTRADA))

for i in range(N_VALIDACION):
    X_validacion_lstm[i, :, :] = X_val_lstm[i:i + VENTANA_TIEMPO]

# Preparación de datos de testeo con ventana temporal
X_testeo_lstm = np.zeros((N_TESTEO, VENTANA_TIEMPO, DIMENSION_ENTRADA))

for i in range(N_TESTEO):
    X_testeo_lstm[i, :, :] = X_test_lstm[i:i + VENTANA_TIEMPO]

In [10]:
modelos = {
    'MLP': MLPRegressor(
        activation='relu',
        alpha=0.01,
        hidden_layer_sizes=(64,),
        learning_rate='constant',
        solver='sgd'
    ),
    'Linear Regression': MultiOutputRegressor(LinearRegression()),
    'Elastic Net': MultiOutputRegressor(ElasticNet(
        alpha=0.1,
        l1_ratio=0.5
    )),
    'Decision Tree': MultiOutputRegressor(DecisionTreeRegressor(
        max_depth=5,
        min_samples_leaf=3,
        min_samples_split=3
    )),
    'Random Forest': MultiOutputRegressor(RandomForestRegressor(
        max_depth=None,
        min_samples_leaf=3,
        min_samples_split=2,
        n_estimators=100
    )),
    'Gradient Boosting': MultiOutputRegressor(GradientBoostingRegressor(
        learning_rate=0.1,
        max_depth=3,
        n_estimators=50
    )),
    'XGBoost': MultiOutputRegressor(XGBRegressor(
        learning_rate=0.1,
        max_depth=3,
        n_estimators=50
    )),
    'SVR': MultiOutputRegressor(SVR(
        C=0.1,
        epsilon=0.2,
        kernel='linear'
    )),
}

In [11]:
import os
# Cargar modelo LSTM
modelo_lstm = LSTM(18,512,1,3)
modelo_lstm = torch.load(os.path.join(os.path.join(os.getcwd()),'modelo_completo.pth'))
modelo_lstm.eval()
print("Modelo cargado correctamente.")

Modelo cargado correctamente.


In [13]:
# Diccionario para almacenar los resultados
resultados_modelos = []

# Recorrer los modelos
for nombre, modelo in modelos.items():
    print(f"Entrenando el modelo: {nombre}...")
    
    # Ajustar el modelo a los datos de entrenamiento
    modelo.fit(X_entrenamiento.reshape(N_ENTRENAMIENTO, -1), y_entrenamiento)
    
    # Realizar predicciones en los datos de entrenamiento, validación y test
    y_train_pred = modelo.predict(X_entrenamiento.reshape(N_ENTRENAMIENTO, -1))
    y_val_pred = modelo.predict(X_validacion.reshape(N_VALIDACION, -1))
    y_test_pred = modelo.predict(X_testeo.reshape(N_TESTEO, -1))
    
    # Calcular RMSE y MAE para cada conjunto de datos
    train_rmse = np.sqrt(mean_squared_error(y_entrenamiento, y_train_pred))
    val_rmse = np.sqrt(mean_squared_error(y_validacion, y_val_pred))
    test_rmse = np.sqrt(mean_squared_error(y_testeo, y_test_pred))
    
    train_mae = mean_absolute_error(y_entrenamiento, y_train_pred)
    val_mae = mean_absolute_error(y_validacion, y_val_pred)
    test_mae = mean_absolute_error(y_testeo, y_test_pred)
    
    # Calcular MAPE para cada conjunto de datos, manejando divisiones por cero
    train_mape = np.mean(np.abs((y_entrenamiento - y_train_pred) / y_entrenamiento)) * 100
    val_mape = np.mean(np.abs((y_validacion - y_val_pred) / y_validacion)) * 100
    test_mape = np.mean(np.abs((y_testeo - y_test_pred) / y_testeo)) * 100
    
    # Calcular el error máximo para cada conjunto de datos
    train_max_error = np.max(np.abs(y_entrenamiento - y_train_pred)) if len(y_entrenamiento) > 0 else 0
    val_max_error = np.max(np.abs(y_validacion - y_val_pred)) if len(y_validacion) > 0 else 0
    test_max_error = np.max(np.abs(y_testeo - y_test_pred)) if len(y_testeo) > 0 else 0
    
    # Calcular la desviación estándar del error (dispersión) para cada conjunto de datos
    train_error_std = np.std(y_entrenamiento - y_train_pred) if len(y_entrenamiento) > 0 else 0
    val_error_std = np.std(y_validacion - y_val_pred) if len(y_validacion) > 0 else 0
    test_error_std = np.std(y_testeo - y_test_pred) if len(y_testeo) > 0 else 0
    
    # Almacenar los resultados
    resultados_modelos.append({
        'Modelo': nombre,
        'Train RMSE': train_rmse,
        'Val RMSE': val_rmse,
        'Test RMSE': test_rmse,
        'Train MAE': train_mae,
        'Val MAE': val_mae,
        'Test MAE': test_mae,
        'Train MAPE': train_mape,
        'Val MAPE': val_mape,
        'Test MAPE': test_mape,
        'Train Max Error': train_max_error,
        'Val Max Error': val_max_error,
        'Test Max Error': test_max_error,
        'Train Error Std': train_error_std,
        'Val Error Std': val_error_std,
        'Test Error Std': test_error_std
    })

Entrenando el modelo: MLP...
Entrenando el modelo: Linear Regression...
Entrenando el modelo: Elastic Net...
Entrenando el modelo: Decision Tree...
Entrenando el modelo: Random Forest...
Entrenando el modelo: Gradient Boosting...
Entrenando el modelo: XGBoost...
Entrenando el modelo: SVR...


In [14]:
print("Entrenando el modelo: LSTM...")
modelo_lstm.eval() 

with torch.no_grad():  # Desactiva gradientes para evaluación
    # Predicciones para los conjuntos de entrenamiento, validación y test
    y_train_pred = modelo_lstm(torch.tensor(X_entrenamiento_lstm, dtype=torch.float32)).squeeze(-1).numpy()
    y_val_pred = modelo_lstm(torch.tensor(X_validacion_lstm, dtype=torch.float32)).squeeze(-1).numpy()
    y_test_pred = modelo_lstm(torch.tensor(X_testeo_lstm, dtype=torch.float32)).squeeze(-1).numpy()

# Calcular métricas
train_rmse = np.sqrt(mean_squared_error(y_entrenamiento, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_validacion, y_val_pred))
test_rmse = np.sqrt(mean_squared_error(y_testeo, y_test_pred))

train_mae = mean_absolute_error(y_entrenamiento, y_train_pred)
val_mae = mean_absolute_error(y_validacion, y_val_pred)
test_mae = mean_absolute_error(y_testeo, y_test_pred)

# Calcular MAPE 
train_mape = np.mean(np.abs((y_entrenamiento - y_train_pred) / y_entrenamiento)) * 100
val_mape = np.mean(np.abs((y_validacion - y_val_pred) / y_validacion)) * 100
test_mape = np.mean(np.abs((y_testeo - y_test_pred) / y_testeo)) * 100

# Calcular error máximo
train_max_error = np.max(np.abs(y_entrenamiento - y_train_pred)) 
val_max_error = np.max(np.abs(y_validacion - y_val_pred)) 
test_max_error = np.max(np.abs(y_testeo - y_test_pred))

# Calcular la desviación estándar del error (dispersión)
train_error_std = np.std(y_entrenamiento - y_train_pred) 
val_error_std = np.std(y_validacion - y_val_pred)
test_error_std = np.std(y_testeo - y_test_pred)

# Almacenar los resultados en el diccionario
resultados_modelos.append({
    'Modelo': 'LSTM',
    'Train RMSE': train_rmse,
    'Val RMSE': val_rmse,
    'Test RMSE': test_rmse,
    'Train MAE': train_mae,
    'Val MAE': val_mae,
    'Test MAE': test_mae,
    'Train MAPE': train_mape,
    'Val MAPE': val_mape,
    'Test MAPE': test_mape,
    'Train Max Error': train_max_error,
    'Val Max Error': val_max_error,
    'Test Max Error': test_max_error,
    'Train Error Std': train_error_std,
    'Val Error Std': val_error_std,
    'Test Error Std': test_error_std
})

In [15]:
df = pd.DataFrame(resultados_modelos)
df = df.round(3)
df = pd.concat([df[df['Modelo'] == 'LSTM'], df[df['Modelo'] != 'LSTM']], ignore_index=True)

df

,Modelo,Train RMSE,Val RMSE,Test RMSE,Train MAE,Val MAE,Test MAE,Train MAPE,Val MAPE,Test MAPE,Train Max Error,Val Max Error,Test Max Error,Train Error Std,Val Error Std,Test Error Std
0,LSTM,0.881,1.582,1.176,0.654,1.175,0.859,2.707,4.826,3.616,5.336,6.285,5.011,0.881,1.575,1.161
1,MLP,0.576,1.710,1.323,0.437,1.346,0.990,1.855,5.499,4.173,3.359,5.969,5.466,0.575,1.703,1.318
2,Linear Regression,0.762,1.532,1.220,0.579,1.138,0.886,2.400,4.697,3.737,4.384,5.739,5.375,0.762,1.531,1.214
3,Elastic Net,0.881,1.485,1.122,0.656,1.101,0.803,2.719,4.565,3.372,5.623,6.012,5.248,0.881,1.484,1.118
4,Decision Tree,0.852,1.671,1.224,0.647,1.216,0.874,2.687,5.006,3.668,4.251,6.479,6.071,0.852,1.667,1.210
5,Random Forest,0.416,1.558,1.139,0.287,1.114,0.817,1.188,4.620,3.422,3.363,6.340,5.225,0.416,1.558,1.129
6,Gradient Boosting,0.746,1.536,1.132,0.566,1.125,0.797,2.345,4.648,3.329,3.751,6.290,5.335,0.746,1.534,1.129
7,XGBoost,0.754,1.542,1.122,0.566,1.126,0.796,2.347,4.650,3.323,4.314,6.228,5.446,0.754,1.539,1.118
8,SVR,0.801,1.509,1.229,0.559,1.119,0.888,2.322,4.611,3.741,5.458,5.686,5.352,0.801,1.509,1.226
